# Gene Selection Tutorial 

## 0. Importing packages

In [1]:
import scanpy as sc 
import pandas as pd
import numpy as np
import nvr

Please note that this tutorial utilizes Scanpy. Which was described by [Wolf et al., 2018.](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-017-1382-0) Scanpy is a python package for the organization and analysis of large scale scRNA-seq data. Scanpy documentation is available [here](https://scanpy.readthedocs.io/en/stable/).

## 1. Loading and pre-processing raw count data

### Reading h5ad from file

First, we must load the example dataset. This dataset is derived from [GSE102698](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE102698). Here, we provide it structured as a compressed AnnData object available [here](https://github.com/KenLauLab/pCreode/blob/master/data/s1_counts.h5ad). Scanpy's loading functions allow us to quickly load large datasets into our python environment.

In [2]:
adata = sc.read_h5ad("../data/s1_counts.h5ad")
adata

AnnData object with n_obs × n_vars = 1597 × 25507 

This dataset consists of 1597 cells and 25507 genes.

### Optional first pass filtering

This step is an optional filtering step which removes genes based off of very general criteria, such as the number of cells it is seen expressed in or some cumululative number of counts. 

In [3]:
sc.pp.filter_genes(adata,min_cells = 15) #the use of this function removes any gene that is expressed in fewer than 15 cells 

__Note that this tutorial does not make use of this optional first pass filtering__

### Normalization and transformation

Next, we must normalize the dataset and transform it for downstream analyses. We do this through scanpy's normalize_total function, which, given the below parameters, normalizes each gene count to the total number of counts found in that respective cell. Subsequently, we transform these values with nvr's arcsinh_transform() function. Alternative transformation procedures include log based methods.  

In [4]:
sc.pp.normalize_total(adata,max_fraction = 0, target_sum = 1) #normalize by total counts per cell
nvr.arcsinh_transform(adata) #transform through arcsinh

## 2. Unsupervised feature selection

With the data loaded, normalized, and transformed we can now proceed to feature selection. This function will return a AnnData object which contains additional information regarding nvr feature selected genes. This process should only take a few seconds (5-10 seconds on an i5-8259U). 

In [5]:
adata = nvr.nvr_feature_select(adata)

1 -neighbor(s) results in 1597 component(s)
2 -neighbor(s) results in 22 component(s)
3 -neighbor(s) results in 2 component(s)
4 -neighbor(s) results in 1 component(s)
Using minimally connected KNN graph with 4 neighbors
Calculating neighborhood MSE
Calculating global/neighborhood variance ratio
Feature selection complete, 5 seconds elapsed


The output boolean mask can be accessed through standard AnnData conventions. Each "True" in this boolean mask represesnts a selected gene.

In [6]:
adata.uns['NVR_genes']

array([False, False, False, ..., False,  True, False])

Finally, we can view the names of the genes selected by applying the generated mask.

In [7]:
adata.var[adata.uns['NVR_genes']].head(20) #only visualizing the first 20 genes selected

""
index
1500004A13Rik
1810020O05Rik
1810065E05Rik
2010109I03Rik
2200002D01Rik
2310079G19Rik
2810417H13Rik
4931406C07Rik
5330417C22Rik


We can also subset the original AnnData object to generated a feature selected dataset, instead of just looking at the gene names. 

In [9]:
feature_selected_adata = adata[:,adata.uns['NVR_genes']]
feature_selected_adata

View of AnnData object with n_obs × n_vars = 1597 × 414 
    uns: 'neighbors', 'Global_Variance', 'Summed_MSE', 'Edges_per_neighborhood', 'NVR_genes'
    obsm: 'Neighborhood_MSE'